<a href="https://colab.research.google.com/github/w-okada/voice-changer/blob/master/Realtime_Voice_Changer_on_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### [w-okada's Voice Changer](https://github.com/w-okada/voice-changer) | **Colab**

---

## **⬇ VERY IMPORTANT ⬇**

You can use the following settings for better results:

If you're using a index: `f0: RMVPE_ONNX | Chunk: 112 or higher | Extra: 8192`<br>
If you're not using a index: `f0: RMVPE_ONNX | Chunk: 96 or higher | Extra: 16384`<br>
**Don't forget to select a T4 GPU in the GPU field, <b>NEVER</b> use CPU!
> Seems that PTH models performance better than ONNX for now, you can still try ONNX models and see if it satisfies you


*You can always [click here](https://github.com/YunaOneeChan/Voice-Changer-Settings) to check if these settings are up-to-date*

---

### <font color=red>⬇ Always use Colab GPU! (**IMPORTANT!**) ⬇</font>
You need to use a Colab GPU so the Voice Changer can work faster and better\
Use the menu above and click on **Runtime** » **Change runtime** » **Hardware acceleration** to select a GPU (**T4 is the free one**)

---
**Credits**<br>
Realtime Voice Changer by [w-okada](https://github.com/w-okada)<br>
Notebook files updated by [rafacasari](https://github.com/Rafacasari)<br>
Recommended settings by [YunaOneeChan](https://github.com/YunaOneeChan)

**Need help?** [AI Hub Discord](https://discord.gg/aihub) » ***#help-realtime-vc***

---

In [ ]:
# @title Clone repository and install dependencies
# @markdown This first step will download the latest version of Voice Changer and install the dependencies. **It can take some time to complete.**
%cd /content/

!pip install colorama --quiet
from colorama import Fore, Style
import os

print(f"{Fore.CYAN}> Cloning the repository...{Style.RESET_ALL}")
!git clone https://github.com/w-okada/voice-changer.git --quiet
print(f"{Fore.GREEN}> Successfully cloned the repository!{Style.RESET_ALL}")
%cd voice-changer/server/

print(f"{Fore.CYAN}> Installing libportaudio2...{Style.RESET_ALL}")
!apt-get -y install libportaudio2 -qq

print(f"{Fore.CYAN}> Installing pre-dependencies...{Style.RESET_ALL}")
# Install dependencies that are missing from requirements.txt and pyngrok
!pip install faiss-gpu fairseq pyngrok --quiet
!pip install pyworld --no-build-isolation --quiet
print(f"{Fore.CYAN}> Installing dependencies from requirements.txt...{Style.RESET_ALL}")
!pip install -r requirements.txt --quiet

print(f"{Fore.GREEN}> Successfully installed all packages!{Style.RESET_ALL}")

In [ ]:
# @title Start Server **using ngrok**
# @markdown This cell will start the server, the first time that you run it will download the models, so it can take a while (~1-2 minutes)

# @markdown ---
# @markdown You'll need a ngrok account, but <font color=green>**it's free**</font> and easy to create!
# @markdown ---
# @markdown **1** - Create a <font color=green>**free**</font> account at [ngrok](https://dashboard.ngrok.com/signup) or **login with Google/Github account**\
# @markdown **2** - If you didn't logged in with Google/Github, you will need to **verify your e-mail**!\
# @markdown **3** - Click [this link](https://dashboard.ngrok.com/get-started/your-authtoken) to get your auth token, and place it here:
Token = '' # @param {type:"string"}
# @markdown **4** - *(optional)* Change to a region near to you or keep at United States if increase latency\
# @markdown `Default Region: us - United States (Ohio)`
Region = "us - United States (Ohio)" # @param ["ap - Asia/Pacific (Singapore)", "au - Australia (Sydney)","eu - Europe (Frankfurt)", "in - India (Mumbai)","jp - Japan (Tokyo)","sa - South America (Sao Paulo)", "us - United States (Ohio)"]

#@markdown **5** - *(optional)* Other options:
ClearConsole = True  # @param {type:"boolean"}

# ---------------------------------
# DO NOT TOUCH ANYTHING DOWN BELOW!
# ---------------------------------

%cd /content/voice-changer/server

from pyngrok import conf, ngrok
MyConfig = conf.PyngrokConfig()
MyConfig.auth_token = Token
MyConfig.region = Region[0:2]
#conf.get_default().authtoken = Token
#conf.get_default().region = Region
conf.set_default(MyConfig);

import subprocess, threading, time, socket, urllib.request
PORT = 8000

from pyngrok import ngrok
ngrokConnection = ngrok.connect(PORT)
public_url = ngrokConnection.public_url

from IPython.display import clear_output

def wait_for_server():
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', PORT))
        if result == 0:
            break
        sock.close()
    if ClearConsole:
        clear_output()
    print("--------- SERVER READY! ---------")
    print("Your server is available at:")
    print(public_url)
    print("---------------------------------")

threading.Thread(target=wait_for_server, daemon=True).start()

!python3 MMVCServerSIO.py \
  -p {PORT} \
  --https False \
  --content_vec_500 pretrain/checkpoint_best_legacy_500.pt \
  --content_vec_500_onnx pretrain/content_vec_500.onnx \
  --content_vec_500_onnx_on true \
  --hubert_base pretrain/hubert_base.pt \
  --hubert_base_jp pretrain/rinna_hubert_base_jp.pt \
  --hubert_soft pretrain/hubert/hubert-soft-0d54a1f4.pt \
  --nsf_hifigan pretrain/nsf_hifigan/model \
  --crepe_onnx_full pretrain/crepe_onnx_full.onnx \
  --crepe_onnx_tiny pretrain/crepe_onnx_tiny.onnx \
  --rmvpe pretrain/rmvpe.pt \
  --model_dir model_dir \
  --samples samples.json

ngrok.disconnect(ngrokConnection.public_url)